# Delta Lake and Delta Engine 

## Delta Lake quickstart

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("delta-lake-tutorial") \
    .config("spark.sql.session.timeZone", "Asia/Seoul") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.7.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

In [2]:
from delta.tables import *
spark

In [5]:
# 임시 경로를 삭제합니다
!pwd
!rm -rf /home/jovyan/work/tmp

/home/jovyan/work


### Create Table

In [6]:
data = spark.range(0, 5)
data.write.format("delta").save("tmp/delta-table")

### Read Data

In [8]:
df = spark.read.format("delta").load("tmp/delta-table")
df.show()

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  0|
+---+



### Update Table w/ Overwrite

In [9]:
data = spark.range(5, 10)
data.write.format("delta").mode("overwrite").save("tmp/delta-table")

### Conditional update without overwrite

In [10]:
from delta.tables import *
from pyspark.sql.functions import *

deltaTable = DeltaTable.forPath(spark, "tmp/delta-table")

# Update every even value by adding 100 to it
deltaTable.update(
  condition = expr("id % 2 == 0"),
  set = { "id": expr("id + 100") })

# Delete every even value
deltaTable.delete(condition = expr("id % 2 == 0"))

# Upsert (merge) new data
newData = spark.range(0, 20)

deltaTable.alias("oldData") \
  .merge(
    newData.alias("newData"),
    "oldData.id = newData.id") \
  .whenMatchedUpdate(set = { "id": col("newData.id") }) \
  .whenNotMatchedInsert(values = { "id": col("newData.id") }) \
  .execute()

deltaTable.toDF().show()

+---+
| id|
+---+
|  2|
| 15|
|  5|
|  3|
|  4|
| 13|
| 10|
|  8|
| 19|
|  7|
|  0|
|  6|
| 16|
| 11|
| 18|
|  1|
| 17|
| 14|
|  9|
| 12|
+---+



In [11]:
deltaTable.toDF().sort("id").show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+



### Read older versions of data using time travel

In [8]:
df = spark.read.format("delta").option("versionAsOf", 0).load("tmp/delta-table")
df.show()

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  0|
+---+



### Write a stream of data to a table


In [11]:
streamingDf = spark.readStream.format("rate").load()
stream = streamingDf.selectExpr("value as id").writeStream.format("delta").option("checkpointLocation", "tmp/checkpoint").start("tmp/delta-table")

### Read a stream of changes from a table

In [12]:
stream2 = spark.readStream.format("delta").load("/tmp/delta-table").writeStream.format("console").start()

### 참고 사이트
* [Data Engineer Training](https://github.com/psyoblade/data-engineer-training)
* [SparkSQL Magic](https://github.com/cryeo/sparksql-magic)